In [52]:
import os
import csv
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [2]:
import pandas as pd
import os

data_dir = os.getcwd() + "/DS_Dataset/DS_Dataset"


for i in range(1, 18):
    file_name = f"L{i}_Train.csv"
    file_path = os.path.join(data_dir, file_name)
    

    df = pd.read_csv(file_path)

    df['DateTime'] = pd.to_datetime(df['DateTime'])
    
    all_dates = df['DateTime'].dt.date
    valid_hours = df[(df['DateTime'].dt.hour >= 9) & (df['DateTime'].dt.hour < 17)]
    valid_dates = set(valid_hours['DateTime'].dt.date)

    no_data_dates = set(all_dates) - valid_dates
    has_data_dates = set(all_dates).intersection(valid_dates)
    
    no_data_df = df[df['DateTime'].dt.date.isin(no_data_dates)]
    has_data_df = df[df['DateTime'].dt.date.isin(has_data_dates)]
    
    output_dir = os.getcwd() + "/DS_Dataset/"
    no_data_df.to_csv(f"{output_dir}L{i}_no_data_days.csv", index=False)
    has_data_df.to_csv(f"{output_dir}L{i}_has_data_days.csv", index=False)


In [54]:
import pandas as pd
import numpy as np
import os

def read_data(file_path):
    df = pd.read_csv(file_path)
    df['DateTime'] = pd.to_datetime(df['DateTime'])
    return df

def avg_data_in_batches(data, batch_size=10):
    avg_data = []
    for i in range(0, len(data), batch_size):
        batch = data.iloc[i:i + batch_size]
        avg = batch[['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)']].mean().round(2).values
        avg_data.append(avg)
    return np.array(avg_data)

def preprocess_data(df):
    grouped = df.groupby(df['DateTime'].dt.date)
    final_data = []

    for date, group in grouped:
        group_sorted = group.sort_values(by='DateTime')
        
        before_9am = group_sorted[group_sorted['DateTime'].dt.hour < 9]
        before_9am_avg = []
        if before_9am.empty:
            continue

        if len(before_9am) > 0:
            idx = before_9am.index[-1]  
            for _ in range(12):
                batch = before_9am.loc[idx-10:idx] if idx - 10 >= before_9am.index[0] else before_9am.loc[before_9am.index[0]:idx]
                avg = batch[['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)']].mean().round(2).values
                before_9am_avg.append(avg)
                if idx - 10 >= before_9am.index[0]:
                    idx -= 10
                else:
                    break

            while len(before_9am_avg) < 12:
                before_9am_avg.append(before_9am_avg[-1])

            before_9am_avg = before_9am_avg[::-1]

        final_data.append((np.array(before_9am_avg), np.zeros(len(before_9am_avg), dtype=int)))

        after_9am = group_sorted[(group_sorted['DateTime'].dt.hour >= 9) & (group_sorted['DateTime'].dt.hour < 17)]
        after_9am_avg = avg_data_in_batches(after_9am, batch_size=10)

        if len(after_9am_avg) >= 48:
            after_9am_avg = after_9am_avg[:48]

        final_data.append((after_9am_avg, np.ones(len(after_9am_avg), dtype=int)))

    return final_data

def save_to_csv(final_data, output_file):
    all_data = []
    for data, labels in final_data:
        for i in range(len(data)):
            all_data.append(np.concatenate([data[i], [labels[i]]]))
    
    df = pd.DataFrame(all_data, columns=['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)', 'Label'])
    df.to_csv(output_file, index=False)

def process_files():
    for i in range(1, 18):
        data_file_path = os.getcwd() + f"/DS_Dataset/L{i}_has_data_days.csv"  
        output_file_path = os.getcwd() + f"/DS_Dataset/L{i}_processed_data_with_labels.csv" 

        df = read_data(data_file_path)  
        final_data = preprocess_data(df)  
        save_to_csv(final_data, output_file_path)  

        print(f"L{i} 数据已成功保存至 {output_file_path}")

process_files()


L1 数据已成功保存至 c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/L1_processed_data_with_labels.csv
L2 数据已成功保存至 c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/L2_processed_data_with_labels.csv
L3 数据已成功保存至 c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/L3_processed_data_with_labels.csv
L4 数据已成功保存至 c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/L4_processed_data_with_labels.csv
L5 数据已成功保存至 c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/L5_processed_data_with_labels.csv
L6 数据已成功保存至 c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/L6_processed_data_with_labels.csv
L7 数据已成功保存至 c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/L7_processed_data_with_labels.csv
L8 数据已成功保存至 c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/L8_processed_data_with_labels.csv
L9 数据已成功保存至 c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/L9_processed_data_with_labels.csv
L10 数据已成功保存至 c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/L10_processed_data_with_labels.csv
L11 数据已成功保存至 c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/L11_processed_data_with_labels

In [55]:
import csv
import os

input_file = os.getcwd() + "/DS_Dataset/test.csv" 
output_file = os.getcwd() + "/DS_Dataset/output.csv" 

def process_data(input_file, output_file):
    try:
        with open(input_file, 'r', encoding='utf-8') as file:
            data = file.readlines()

        results = []
        for line in data:
            line = line.strip()  
            if len(line) == 15: 
                date_part = line[4:8]  
                tag_part = line[-3:-1]  
                results.append([date_part, tag_part])
            else:
                print(f"跳過行: {line}")  

        

        with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:  
            writer = csv.writer(csvfile)
            writer.writerow(['Date_Part', 'Tag_Part'])
            writer.writerows(results)

        print(f"資料已成功處理並寫入到 {output_file} 中！")
    
process_data(input_file, output_file)




跳過行: 序號,答案
資料已成功處理並寫入到 c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/output.csv 中！


In [62]:
import csv
import os
from datetime import datetime, timedelta
from collections import defaultdict

output_csv = os.getcwd() + "/DS_Dataset/output.csv"
L_files_folder = os.getcwd() + "/DS_Dataset/"
L_files = [L_files_folder + f"L{i}_no_data_days.csv" for i in range(1, 18)]
result_csv = os.getcwd() + "/DS_Dataset/no_data.csv"

def read_output(output_csv):
    """從 output.csv 讀取 Date_Part 和 Tag_Part"""
    date_tag_map = defaultdict(list)
    with open(output_csv, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            date_part = row['Date_Part']
            tag_part = int(row['Tag_Part'])
            date_tag_map[tag_part].append(date_part)
    return date_tag_map

def process_L_files(L_files, date_tag_map, result_csv):
    """處理 L 檔案，針對每一天從 09:00:00 往前取 12 個 10 分鐘平均的數據，並按時間遞增排序"""
    result_data = []

    for tag, dates in date_tag_map.items():
        L_file = L_files[int(tag) - 1]
        if not os.path.exists(L_file):
            print(f"檔案 {L_file} 不存在，跳過處理。")
            continue

        with open(L_file, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            temp_data = defaultdict(list)

            for row in reader:
                datetime_str = row['DateTime']
                record_time = datetime.strptime(datetime_str, '%Y-%m-%d %H:%M:%S')
                date_str = record_time.strftime('%m%d')

                if date_str not in dates:
                    continue

                temp_data[record_time.date()].append({
                    'time': record_time,
                    'WindSpeed(m/s)': float(row['WindSpeed(m/s)']),
                    'Pressure(hpa)': float(row['Pressure(hpa)']),
                    'Temperature(°C)': float(row['Temperature(°C)']),
                    'Humidity(%)': float(row['Humidity(%)']),
                    'Sunlight(Lux)': float(row['Sunlight(Lux)']),
                    'Power(mW)': float(row['Power(mW)']),
                })

            for date, records in temp_data.items():
                filtered_data = [rec for rec in records if rec['time'] <= datetime.combine(date, datetime.min.time()) + timedelta(hours=9)]
                filtered_data.sort(key=lambda x: x['time'], reverse=True)

                interval_data = defaultdict(lambda: defaultdict(list))
                for rec in filtered_data:
                    interval_time = rec['time'].replace(minute=(rec['time'].minute // 10) * 10, second=0)
                    interval_data[interval_time]['WindSpeed(m/s)'].append(rec['WindSpeed(m/s)'])
                    interval_data[interval_time]['Pressure(hpa)'].append(rec['Pressure(hpa)'])
                    interval_data[interval_time]['Temperature(°C)'].append(rec['Temperature(°C)'])
                    interval_data[interval_time]['Humidity(%)'].append(rec['Humidity(%)'])
                    interval_data[interval_time]['Sunlight(Lux)'].append(rec['Sunlight(Lux)'])
                    interval_data[interval_time]['Power(mW)'].append(rec['Power(mW)'])

                averaged_intervals = []
                for interval, metrics in interval_data.items():
                    averaged_intervals.append({
                        'Interval': interval,
                        'WindSpeed(m/s)': round(sum(metrics['WindSpeed(m/s)']) / len(metrics['WindSpeed(m/s)']), 2),
                        'Pressure(hpa)': round(sum(metrics['Pressure(hpa)']) / len(metrics['Pressure(hpa)']), 2),
                        'Temperature(°C)': round(sum(metrics['Temperature(°C)']) / len(metrics['Temperature(°C)']), 2),
                        'Humidity(%)': round(sum(metrics['Humidity(%)']) / len(metrics['Humidity(%)']), 2),
                        'Sunlight(Lux)': round(sum(metrics['Sunlight(Lux)']) / len(metrics['Sunlight(Lux)']), 2),
                        'Power(mW)': round(sum(metrics['Power(mW)']) / len(metrics['Power(mW)']), 2),
                        'Label': f"L{tag}"
                    })

                averaged_intervals.sort(key=lambda x: x['Interval'], reverse=True)
                if len(averaged_intervals) > 12:
                    averaged_intervals = averaged_intervals[:12]
                averaged_intervals.reverse()

                result_data.extend(averaged_intervals)

    with open(result_csv, 'w', newline='', encoding='utf-8') as file:
        fieldnames = ['Interval', 'WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)',
                      'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)', 'Label']
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(result_data)

    print(f"結果已寫入 {result_csv}")

date_tag_map = read_output(output_csv)
process_L_files(L_files, date_tag_map, result_csv)


結果已寫入 c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/no_data.csv


In [113]:

import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt

data_dir = os.getcwd() + "/DS_Dataset/"
for i in range(1, 18):
    file_path = os.path.join(data_dir, f"L{i}_processed_data_with_labels.csv")
    
    df = pd.read_csv(file_path)

    input_features = ['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)']
    output_feature = 'Power(mW)'

    scaler_input = MinMaxScaler()
    scaler_output = MinMaxScaler()

    scaled_inputs = scaler_input.fit_transform(df[input_features])
    scaled_output = scaler_output.fit_transform(df[[output_feature]])

    df_scaled = pd.DataFrame(scaled_inputs, columns=input_features)
    df_scaled[output_feature] = scaled_output
    df_scaled['Label'] = df['Label']

    sequence_length = 12
    prediction_horizon = 48

    input_data = df_scaled[df_scaled['Label'] == 0][input_features].values
    output_data = df_scaled[df_scaled['Label'] == 1][output_feature].values

    X, y = [], []
    for j in range(len(input_data) - sequence_length - prediction_horizon):
        X.append(input_data[j:j + sequence_length])
        y.append(output_data[j + sequence_length:j + sequence_length + prediction_horizon])

    X = np.array(X)
    y = np.array(y)

    split_index = int(len(X) * 0.8)
    X_train, X_test = X[:split_index], X[split_index:]
    y_train, y_test = y[:split_index], y[split_index:]

    regressor = Sequential()
    regressor.add(LSTM(units=128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    regressor.add(LSTM(units=64))
    regressor.add(Dropout(0.2))
    regressor.add(Dense(units=48))
    regressor.compile(optimizer='adam', loss='mean_squared_error')

    history = regressor.fit(X_train, y_train, epochs=100, batch_size=320, validation_data=(X_test, y_test))

    model_save_path = os.path.join(data_dir, f'lstm_power_model_L{i}.h5')  
    regressor.save(model_save_path) 
    print(f'Model for L{i} saved at {model_save_path}')



Epoch 1/100


c:\Users\djes1\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 270ms/step - loss: 0.1165 - val_loss: 0.1692
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.0997 - val_loss: 0.1469
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.0917 - val_loss: 0.1432
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.0885 - val_loss: 0.1426
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - loss: 0.0870 - val_loss: 0.1413
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0867 - val_loss: 0.1450
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 0.0846 - val_loss: 0.1461
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0851 - val_loss: 0.1438
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0834 - val_loss: 0.1457
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.0845 - val_loss: 0.1426
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0830 - val_loss: 0.1425
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0841 - val_loss: 0.138

Model for L1 saved at c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/lstm_power_model_L1.h5


c:\Users\djes1\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

Epoch 1/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 693ms/step - loss: 0.0766 - val_loss: 0.1388
Epoch 2/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 96ms/step - loss: 0.0697 - val_loss: 0.1281
Epoch 3/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step - loss: 0.0652 - val_loss: 0.1224
Epoch 4/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - loss: 0.0623 - val_loss: 0.1193
Epoch 5/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step - loss: 0.0599 - val_loss: 0.1158
Epoch 6/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 106ms/step - loss: 0.0588 - val_loss: 0.1145
Epoch 7/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.0572 - val_loss: 0.1147
Epoch 8/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 0.0550 - val_loss: 0.1155
Epoch 9/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 132ms/step - loss: 0.0542 - val_loss: 0.1148
Epoch 10/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step - loss: 0.0532 - val_loss: 0.1134
Epoch 11/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - loss: 0.0518 - val_loss: 0.1125
Epoch 12/100
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 155ms/step - loss: 0.0510

Model for L16 saved at c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/lstm_power_model_L16.h5


c:\Users\djes1\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 6s 318ms/step - loss: 0.0556 - val_loss: 0.0754
Epoch 2/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.0483 - val_loss: 0.0706
Epoch 3/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 0.0474 - val_loss: 0.0707
Epoch 4/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.0464 - val_loss: 0.0729
Epoch 5/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 103ms/step - loss: 0.0456 - val_loss: 0.0738
Epoch 6/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step - loss: 0.0451 - val_loss: 0.0757
Epoch 7/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 100ms/step - loss: 0.0445 - val_loss: 0.0772
Epoch 8/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - loss: 0.0448 - val_loss: 0.0788
Epoch 9/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step - loss: 0.0457 - val_loss: 0.0813
Epoch 10/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - loss: 0.0443 - val_loss: 0.0818
Epoch 11/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 97ms/step - loss: 0.0455 - val_loss: 0.0829
Epoch 12/100
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step - loss: 0.0430 -

Model for L17 saved at c:\Users\djes1\Desktop\DS_Dataset/DS_Dataset/lstm_power_model_L17.h5


In [79]:
import pandas as pd
import os

data_dir = os.getcwd() + "/DS_Dataset/"
no_data_path = os.path.join(data_dir, "no_data.csv")

no_data_df = pd.read_csv(no_data_path)

no_data_df['Date'] = pd.to_datetime(no_data_df['Interval']).dt.date
for i in range(1, 18):
    no_data_filtered = no_data_df[no_data_df['Label'] == f"L{i}"]

    insufficient_data_dates = []

    for date, group in no_data_filtered.groupby('Date'):
        if len(group) < 12:
            insufficient_data_dates.append((date, len(group))) 

    if insufficient_data_dates:
        print(f"Dates with insufficient data for Label {i}:")
        for date, count in insufficient_data_dates:
            print(f"Date: {date}, Records: {count}")
    else:
        print(f"All dates have sufficient data for Label {i}.")


All dates have sufficient data for Label 1.
All dates have sufficient data for Label 2.
All dates have sufficient data for Label 3.
All dates have sufficient data for Label 4.
All dates have sufficient data for Label 5.
All dates have sufficient data for Label 6.
All dates have sufficient data for Label 7.
All dates have sufficient data for Label 8.
All dates have sufficient data for Label 9.
All dates have sufficient data for Label 10.
All dates have sufficient data for Label 11.
All dates have sufficient data for Label 12.
All dates have sufficient data for Label 13.
All dates have sufficient data for Label 14.
All dates have sufficient data for Label 15.
All dates have sufficient data for Label 16.
All dates have sufficient data for Label 17.


In [100]:
import pandas as pd
import os

# 读取 no_data.csv 文件
no_data = pd.read_csv(os.path.join(data_dir, "no_data.csv"))

# 获取所有不同的 Label 值
labels = no_data['Label'].unique()

# 针对每个 Label 创建一个新的 CSV 文件
for label in labels:
    # 根据 Label 分割数据
    label_data = no_data[no_data['Label'] == label]
    
    # 创建新的文件名
    label_filename = f"no_data_label_{label}.csv"
    label_filepath = os.path.join(data_dir, label_filename)
    
    # 将数据保存为 CSV 文件
    label_data.to_csv(label_filepath, index=False)
    print(f"Saved {label_filename}")


Saved no_data_label_L1.csv
Saved no_data_label_L2.csv
Saved no_data_label_L3.csv
Saved no_data_label_L4.csv
Saved no_data_label_L5.csv
Saved no_data_label_L6.csv
Saved no_data_label_L7.csv
Saved no_data_label_L8.csv
Saved no_data_label_L9.csv
Saved no_data_label_L10.csv
Saved no_data_label_L11.csv
Saved no_data_label_L12.csv
Saved no_data_label_L13.csv
Saved no_data_label_L14.csv
Saved no_data_label_L15.csv
Saved no_data_label_L16.csv
Saved no_data_label_L17.csv


In [115]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

data_dir = "./DS_Dataset"

columns_to_read = ['WindSpeed(m/s)', 'Pressure(hpa)', 'Temperature(°C)', 'Humidity(%)', 'Sunlight(Lux)', 'Power(mW)']

no_data_L1 = pd.read_csv(os.path.join(data_dir, "no_data_label_L1.csv"), usecols=columns_to_read)
L1_processed_data = pd.read_csv(os.path.join(data_dir, "L1_processed_data_with_labels.csv"), usecols=columns_to_read)

combined_data = pd.concat([no_data_L1, L1_processed_data], axis=0, ignore_index=True)

input_features = combined_data.iloc[:, :-1]
output_feature = combined_data.iloc[:, -1]

input_scaler = MinMaxScaler()
output_scaler = MinMaxScaler()

input_scaler.fit(input_features)
output_scaler.fit(output_feature.values.reshape(-1, 1))

no_data_L1_inputs_scaled = input_scaler.transform(no_data_L1.iloc[:, :-1])

time_steps = 12
n_samples = len(no_data_L1_inputs_scaled)
n_batches = n_samples // time_steps
batches = no_data_L1_inputs_scaled.reshape(n_batches, time_steps, no_data_L1_inputs_scaled.shape[1])

model = load_model("./DS_Dataset/lstm_power_models_L1.h5")

predictions = []
for batch in batches:
    batch_predictions_scaled = model.predict(batch[np.newaxis, :, :])  # 增加 batch_size 维度
    batch_predictions_inverse = output_scaler.inverse_transform(batch_predictions_scaled)
    predictions.append(batch_predictions_inverse)

predictions_array = np.vstack(predictions)
print(predictions_array.shape)
#predictions_array
#predictions_df = pd.DataFrame(predictions_array, columns=['Predicted Power(mW)'])
#output_csv_path = os.path.join(data_dir, "no_data_L1_predictions.csv")
#predictions_df.to_csv(output_csv_path, index=False)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 464ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
(12, 48)


array([[ 137.45499   ,  145.19781   ,  133.9125    ,   93.586914  ,
         168.22673   ,  127.556694  ,   91.51155   ,  -73.686134  ,
        -107.57934   ,   92.76346   ,    3.8170986 ,  119.524345  ,
          98.42088   ,    2.801028  ,  139.80563   ,  -55.25264   ,
         118.96587   ,   73.451385  ,    9.097268  ,   82.12263   ,
          -7.4783425 ,   98.250656  ,   45.944508  ,   47.26578   ,
         114.90002   ,  -43.690483  ,  140.29247   ,  102.1178    ,
          84.89215   ,   43.092575  ,   23.073174  ,   27.639194  ,
         150.04881   , -127.75672   , -140.3722    ,   12.285505  ,
         -78.77692   ,  134.06583   ,  178.75945   ,  -22.47952   ,
           7.2490444 ,  154.17693   ,  -91.756874  ,  103.35628   ,
          11.65419   ,   18.460457  ,    0.49001974,  -25.309935  ],
       [ 158.05812   ,  180.7446    ,  151.34706   ,  112.36168   ,
         202.10913   ,  139.17603   ,  108.549576  ,  -90.456795  ,
        -138.58246   ,   78.781105  ,    1.7323